## Import Modules

In [ ]:
# Import Modules

import dash
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_core_components as dcc
import dash_html_components as html
import dash_extensions as de
import dash_leaflet as dl
from jupyter_dash import JupyterDash
import dash_leaflet.express as dlx
from dash_extensions.javascript import arrow_function
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

# Create App

external_stylesheets = [
    'https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css',
    'https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css',
    'https://codepen.io/chriddyp/pen/bWLwgP.css',
]


app = JupyterDash(
    name=__name__,
    external_stylesheets=external_stylesheets,
    prevent_initial_callbacks=True
)


# Create Layout App


keys = {
    "One": {
        "url": "https://stamen-tiles-{s}.a.ssl.fastly.net/terrain/{z}/{x}/{y}{r}.png",
        "name": "Terrain"
    },
    "Two": {
        "url": "https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
        "name": "Open Street Map"
    },
    "Three": {
        "url": "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}",
        "name": "Google Satellite Imagery"
    }
}

AQUIFERS = "./App/static/shapefiles/Aquifers.geojson"
MAHDOUDE = "./App/static/shapefiles/Mahdoude.geojson"
HOZEH6 = "./App/static/shapefiles/Hozeh6.geojson"
HOZEH30 = "./App/static/shapefiles/Hozeh30.geojson"
OSTAN = "./App/static/shapefiles/Ostan.geojson"
SHAHRESTAN = "./App/static/shapefiles/Shahrestan.geojson"


attribution = '&copy; <a href="http://www.khrw.ir/">Khorasan Regional Water Company</a> '



info = html.Div(
    id="info",
    className="info",
    style={
        "position": "absolute",
        "bottom": "10px",
        "left": "10px",
        "zIndex": "1000"
    },
    dir="rtl"
)


search_bar = html.Div(
    children=[
        dcc.Input(
            id="search_lat_lng",
            type="search",
            placeholder="Lat: 36.30 Lon: 59.60",
            debounce=True
        )
    ],
    style={
        "position": "absolute",
        "top": "10px",
        "left": "50px",
        "zIndex": "1000"
    },
    dir="ltr"
)


model = html.Div(
    children=[
        dbc.Modal(
            id="modal",
            centered=True,
            is_open=False,
            children=[
                dbc.ModalHeader(
                    id="modal_header"
                ),
                dbc.ModalBody(
                    id="modal_body"
                )
            ]
        )
    ]
)


app.layout = html.Div([
    
    html.Div(
        children=[
            
            dl.Map(
                id="map",
#                 center=[36.30, 59.60],
                zoom=5,
                children=[
                    dl.LayersControl(
                        children=[
                            dl.BaseLayer(
                                children=[
                                    dl.TileLayer(
                                        url=key["url"],
                                        attribution=attribution
                                    ),
                                ],
                                name=key["name"],
                                checked=key["name"] == "Open Street Map"  
                            ) for key in keys.values()                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.LayerGroup(
                                        id="layer"
                                    )                                    
                                ],
                                name="click+search",
                                checked=True
                            )                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.GeoJSON(
                                        id="ostan",
                                        url="/assets/Ostan.geojson",
#                                         zoomToBounds=True,
#                                         zoomToBoundsOnClick=True,
                                        hoverStyle=arrow_function(
                                            dict(
                                                weight=5,
                                                color='#222',
                                                dashArray=''
                                            )
                                        ),
                                        options={
                                            "style": {
                                                "color": "blue"
                                            }
                                        }                                       
                                    )                                    
                                ],
                                name="استان",
                                checked=False
                            )                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.GeoJSON(
                                        id="shahrestan",
                                        url="/assets/Shahrestan.geojson",
#                                         zoomToBounds=True,
                                        zoomToBoundsOnClick=True,
                                        hoverStyle=arrow_function(
                                            dict(
                                                weight=5,
                                                color='#222',
                                                dashArray=''
                                            )
                                        ),
                                        options={
                                            "style": {
                                                "color": "#FD8D3C"
                                            }
                                        }                                       
                                    )                                    
                                ],
                                name="شهرستان",
                                checked=False
                            )                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.GeoJSON(
                                        id="hozeh6",
                                        url="/assets/Hozeh6.geojson",
#                                         zoomToBounds=True,
#                                         zoomToBoundsOnClick=True,
                                        hoverStyle=arrow_function(
                                            dict(
                                                weight=5,
                                                color='#222',
                                                dashArray=''
                                            )
                                        ),
                                        options={
                                            "style": {
                                                "color": "red"
                                            }
                                        },
                                    )                                    
                                ],
                                name="حوضه درجه یک",
                                checked=False
                            )                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.GeoJSON(
                                        id="hozeh30",
                                        url="/assets/Hozeh30.geojson",
#                                         zoomToBounds=True,
#                                         zoomToBoundsOnClick=True,
                                        hoverStyle=arrow_function(
                                            dict(
                                                weight=5,
                                                color='#222',
                                                dashArray=''
                                            )
                                        ),
                                        options={
                                            "style": {
                                                "color": "green"
                                            }
                                        }                                       
                                    )                                    
                                ],
                                name="حوضه درجه دو",
                                checked=False
                            )                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.GeoJSON(
                                        id="mahdoude",
                                        url="/assets/Mahdoude.geojson",
                                        zoomToBounds=True,
                                        zoomToBoundsOnClick=True,
                                        hoverStyle=arrow_function(
                                            dict(
                                                weight=5,
                                                color='#222',
                                                dashArray=''
                                            )
                                        ),
                                        options={
                                            "style": {
                                                "color": "tomato"
                                            }
                                        }                                       
                                    )                                    
                                ],
                                name="محدوده مطالعاتی",
                                checked=True
                            )                            
                        ]
                    ),
                    dl.LocateControl(
                        options={
                            'locateOptions': {
                                'enableHighAccuracy': True
                            }
                        }
                    ),
                    dl.MeasureControl(
                        position="topleft",
                        primaryLengthUnit="kilometers",
                        primaryAreaUnit="hectares",
                        activeColor="#214097",
                        completedColor="#972158"
                    ),
                    search_bar,
                    info,
                    model
                ],
                style={
                    'width': '100%', 
                    'height': '75vh',
                    'margin': "auto",
                },
            ),
        ],
        dir="rtl"
    ),
    html.Div(
        children=[
            html.Div(
                id="out1",
                className="col-6"
            ),
            html.Div(
                id="out2",
                className="col-6"
            )
        ],
        className="row"
    )
])


# App Callback

@app.callback(
    Output("layer", "children"),
    Output("search_lat_lng", "value"),
    Output("map", "center"),
    Input("map", "dbl_click_lat_lng"),
    Input("search_lat_lng", "value")
)
def map_dbl_click(dbl_click_lat_lng, search_lat_lng):
    print(search_lat_lng)
   
    if search_lat_lng is None or search_lat_lng == "" and dbl_click_lat_lng:
        result = dl.Marker(
            children=dl.Tooltip(
                "({:.3f}, {:.3f})".format(*dbl_click_lat_lng)
            ),
            position=dbl_click_lat_lng
        )

        return [result], "", dbl_click_lat_lng
    
    else:
        
        search_lat_lng = list(
            filter(
                ("").__ne__,
                search_lat_lng.split(" ")
            )
        )
        
        if len(search_lat_lng) == 2:
        
            search_lat_lng = [float(x) for x in search_lat_lng]

            result = dl.Marker(
                children=dl.Tooltip(
                    "({:.2f}, {:.2f})".format(*search_lat_lng)
                ),
                position=search_lat_lng
            )

            return [result], "", search_lat_lng
        else:
            raise PreventUpdate
    

@app.callback(
    Output("out1", "children"), 
    Input("ostan", "click_feature")
)
def ostan_click(feature):
    if feature is not None:
        return f"شما {feature['properties']['ostn_name']}"
    
@app.callback(
    Output("info", "children"), 
    Input("ostan", "hover_feature"),
    Input("mahdoude", "hover_feature"),
)
def ostan_click(feature_ostan, feature_mahdoude):
    header = [
        html.H4(
            "US Population Density"
        )
    ]
    
    if feature_ostan:
        return header + [
            html.B(feature_ostan["properties"]["ostn_name"]),
            html.Br(),
            "{:.1f} ha".format(feature_ostan["properties"]["AREA"] / 10000)
        ]
    elif feature_mahdoude:
        return header + [
            html.B(f"نام محدوده مطالعاتی: {feature_mahdoude['properties']['MahName']}"),
            html.Br(),
            html.B(f"کد محدوده مطالعاتی: {feature_mahdoude['properties']['MahCode']}"),
            html.Br(),
            html.B(f"حوزه درجه دو: {feature_mahdoude['properties']['Hoze30Name']}"),
                        html.Br(),
            html.B(f"حوزه درجه یک: {feature_mahdoude['properties']['Hoze6Name']}"),
        ]
    else:
        return header + ["Hoover over a state"]


@app.callback(
    Output("ostan", "children"),
    Input("ostan", "hover_feature"),
)
def oss(feature_ostan):
    if feature_ostan is not None:
        return dl.Tooltip(
            f"{feature_ostan['properties']['ostn_name']}"
        )
    else:
        raise PreventUpdate
        

        


df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')

fig = px.scatter(df, x="gdp per capita", y="life expectancy",
                 size="population", color="continent", hover_name="country",
                 log_x=True, size_max=60)        
        

@app.callback(
    Output("modal", "is_open"),
    Output("modal_header", "children"),
    Output("modal_body", "children"),
    Input("ostan", "click_feature"),
    State("modal", "is_open"),
)
def toggle_modal(feature_ostan, is_open):
    if feature_ostan is not None:
        return not is_open, f"{feature_ostan['properties']['ostn_name']}", html.Div([dcc.Graph(id='life-exp-vs-gdp', figure=fig)
])
    else:
        raise PreventUpdate



# Run App

app.run_server(
    mode="external",
)

In [ ]:
import geopandas as gpd
data = gpd.read_file('assets/Hozeh6.geojson')
print(data)

# --------------------------------------------------------------------------------
# Convert Wide Data To Long Data
# --------------------------------------------------------------------------------

In [ ]:
import pandas as pd
import numpy as np

WaterTableData = pd.read_csv("WaterTableData.csv")

ThissenAreaData = pd.read_csv("ThissenAreaData.csv")

waterTable = pd.melt(
    WaterTableData,
    id_vars=WaterTableData.columns[0:4].to_list(),
    var_name='Date',
    value_name='wt'
)

waterTable["Date"] = waterTable["Date"].apply(pd.to_datetime)

thissenArea = pd.melt(
    ThissenAreaData,
    id_vars=ThissenAreaData.columns[0:4].to_list(),
    var_name='Date',
    value_name='th'
)

thissenArea["Date"] = thissenArea["Date"].apply(pd.to_datetime)

result = waterTable.merge(
    thissenArea,
    how="outer",
    on=["Mahdodeh_Name", "Mahdodeh_Code", "Aquifer_Name", "Well_Name", "Date"]
).sort_values(by=["Mahdodeh_Name", "Mahdodeh_Code", "Aquifer_Name", "Well_Name", "Date"])

result.columns = ["MAHDOUDE_NAME", "MAHDOUDE_CODE", "AQUIFER_NAME", "LOCATION_NAME", "DATE", "WATER_TABLE", "THISSEN_LOCATION"]

# Mean Thissen
tm = result.groupby(['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE']).agg({'THISSEN_LOCATION': 'sum'}).reset_index()
tm.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE", "THISSEN_AQUIFER"]

result = result.merge(
    tm,
    how="outer",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE"]
).sort_values(by=["MAHDOUDE_NAME", "MAHDOUDE_CODE", "AQUIFER_NAME", "LOCATION_NAME", "DATE"])

result['THISSEN_AQUIFER'] = result['THISSEN_AQUIFER'].replace(0, np.nan)

result.to_excel("HydrographDataRaw.xlsx")

# Calculate Aquifer Hydrograph

In [ ]:
import pandas as pd
import numpy as np
import jalali

df = pd.read_excel("DATE.xlsx")

df["DATE_GEROGORIAN"] = df["DATE_GEROGORIAN"].apply(pd.to_datetime)

df['DATE_CHECK'] = np.where(
    df["DATE_PERSIAN"].isna(),
    np.where(
        df["DATE_GEROGORIAN"].isna(),
        np.NaN,
        "G"
    ),
    "P"  
)





df['DATE_PERSIAN'] = df.apply(
    lambda x: jalali.Gregorian(x["DATE_GEROGORIAN"].date()).persian_string() if x["DATE_CHECK"] == "G" else x["DATE_PERSIAN"], 
    axis=1
)

df['DATE_GEROGORIAN'] = df.apply(
    lambda x: jalali.Persian(x["DATE_PERSIAN"]).gregorian_string() if x["DATE_CHECK"] == "P" else x["DATE_GEROGORIAN"], 
    axis=1
)


print(df.info())

df

In [1]:
import dash_leaflet as dl
import dash_leaflet.express as dlx
import geojson
import geobuf
import geopandas as gpd
import json

/home/pooya/w/KHRW-DASHBOARD-LOCAL/Flask/.venv/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [ ]:
import geojson
import dash_leaflet.express as dlx

PROVINCE = "../Assets/GeoJson/Basin1.geojson"

with open(PROVINCE) as f:
    data = geojson.load(f)

data = dlx.geojson_to_geobuf(data)

In [ ]:
a = dlx.geojson_to_geobuf(data)

In [4]:
GEOJSON_LOCATION = {
    "BASIN1" : "../Assets/GeoJson/Basin1.geojson",
    "BASIN2" : "../Assets/GeoJson/Basin2.geojson",
    "MAHDOUDE" : "../Assets/GeoJson/Mahdoude.geojson",
    "AQUIFER" : "../Assets/GeoJson/Aquifer.geojson",
    # "COUNTRY" : "./Assets/GeoJson/Country.geojson",
    "PROVINCE" : "../Assets/GeoJson/Province.geojson",
    "COUNTY" : "../Assets/GeoJson/County.geojson",
    "DISTRICT" : "../Assets/GeoJson/District.geojson",
}

list(GEOJSON_LOCATION.keys())

['BASIN1', 'BASIN2', 'MAHDOUDE', 'AQUIFER', 'PROVINCE', 'COUNTY', 'DISTRICT']

In [2]:
GEOJSON_LOCATION = {
    "COUNTRY" : {
        "url": "./Assets/GeoJson/Country.geojson",
        "options": {
            "weight": "1",
            # "dashArray": "10, 10",
            # "dashOffset": "10",
            "color": "black",
            "opacity": "1",
            "fillColor": "black",
            "fillOpacity": "0.01",
        }
    },
    "PROVINCE" : {
        "url": "./Assets/GeoJson/Province.geojson",
        "options": {
            "weight": "1",
            # "dashArray": "10, 10",
            # "dashOffset": "10",
            "color": "black",
            "opacity": "1",
            "fillColor": "black",
            "fillOpacity": "0.01",
        }
    },
    "COUNTY" : {
        "url": "./Assets/GeoJson/County.geojson",
        "options": {
            "weight": "1",
            # "dashArray": "10, 10",
            # "dashOffset": "10",
            "color": "black",
            "opacity": "1",
            "fillColor": "black",
            "fillOpacity": "0.01",
        }
    },
    "DISTRICT" : {
        "url": "./Assets/GeoJson/District.geojson",
        "options": {
            "weight": "1",
            # "dashArray": "10, 10",
            # "dashOffset": "10",
            "color": "black",
            "opacity": "1",
            "fillColor": "black",
            "fillOpacity": "0.01",
        }
    },
    "BASIN1" : {
        "url": "./Assets/GeoJson/Basin1.geojson",
        "options": {
            "weight": "5",
            # "dashArray": "10, 10",
            # "dashOffset": "10",
            "color": "blue",
            "opacity": "1",
            "fillColor": "blue",
            "fillOpacity": "0.01",
        }
    },
    "BASIN2" : {
        "url": "./Assets/GeoJson/Basin2.geojson",
        "options": {
            "weight": "5",
            # "dashArray": "10, 10",
            # "dashOffset": "10",
            "color": "blue",
            "opacity": "1",
            "fillColor": "blue",
            "fillOpacity": "0.01",
        }
    },
    "MAHDOUDE" : {
        "url": "./Assets/GeoJson/Mahdoude.geojson",
        "options": {
            "weight": "5",
            # "dashArray": "10, 10",
            # "dashOffset": "10",
            "color": "blue",
            "opacity": "1",
            "fillColor": "blue",
            "fillOpacity": "0.01",
        }
    },
    "AQUIFER" : {
        "url": "./Assets/GeoJson/Aquifer.geojson",
        "options": {
            "weight": "5",
            # "dashArray": "10, 10",
            # "dashOffset": "10",
            "color": "blue",
            "opacity": "1",
            "fillColor": "blue",
            "fillOpacity": "0.01",
        }
    },
}

list(GEOJSON_LOCATION.keys())


['COUNTRY',
 'PROVINCE',
 'COUNTY',
 'DISTRICT',
 'BASIN1',
 'BASIN2',
 'MAHDOUDE',
 'AQUIFER']

In [3]:
from dash.dependencies import Input, Output, State

map_items = ["COUNTRY", "IRAN"]


def create_inputs_callback(
    data,
    component_id="MAP-TAB_HOME_BODY",
    component_property="hover_feature",
):
    return [
        [Input(f"{i}_{component_id}", component_property) for i in data],
    ]


create_inputs_callback(data=map_items)



[[<Input `COUNTRY_MAP-TAB_HOME_BODY.hover_feature`>,
  <Input `IRAN_MAP-TAB_HOME_BODY.hover_feature`>]]

In [6]:
data = ['AQUIFER']

data[0]

'AQUIFER'